# 5. Evaluation

In [ ]:
# !pip install pandas
# !pip install matplotlib
# !pip install seaborn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Logs Path
path_to_logs = '../Results/'

# Load the logs from CSV files
collected_logs_df = pd.read_csv(path_to_logs + 'Log_Generalization/4.4.1-collected_logs.csv')
templated_logs_df = pd.read_csv(path_to_logs + 'Log_Generalization/4.4.2-generalized_logs.csv')
difference_logs_df = pd.read_csv(path_to_logs + 'Log_Generalization/4.4.3-difference_logs.csv')
generalized_logs_df = pd.read_csv(path_to_logs + 'Log_Generalization/4.4.4-enriched_logs.csv')
determination_logs_df = pd.read_csv(path_to_logs + 'Anomaly_Determination/4.5-determination_logs.csv')

## Help Functions

### Count

In [ ]:
# Count logs by attribute
def count_logs(logs_df, attribute):
    grouped_logs = logs_df.groupby(attribute).size().reset_index(name='count')
    return grouped_logs

# Group logs by 2 attributes and count
def group_by_and_count(logs_df, attribute1,attribute2):
    grouped_logs = logs_df.groupby([attribute1, attribute2]).size().reset_index(name='count')
    return grouped_logs

# Group logs by 2 attributes and sum
def group_by_and_sum(logs_df, attribute1,attribute2):
    grouped_logs = logs_df.groupby([attribute1, attribute2]).sum().reset_index(name='sum')
    return grouped_logs

# Group logs by 3 attributes and count
def group_by_three_and_sum(logs_df, attribute1,attribute2, attribute3):
    grouped_logs = logs_df.groupby([attribute1, attribute2]).agg(frequency_phase_sum=(attribute3, 'sum')).reset_index()
    grouped_logs = grouped_logs.rename(columns={'frequency_phase_sum': 'count'})        
    return grouped_logs

### Plot

In [ ]:
# Generate a flexible plot
def plot_logs_data(logs_df, sortattribute, x_column, y_column=None, plot_type='bar', title=None, pivot=False, agg_func='sum'):
    plt.figure(figsize=(20, 10))
    
    # If pivot is True, create a pivot table to handle grouped data with multiple y values
    if pivot and y_column:
        pivot_df = logs_df.pivot_table(index=x_column, columns=y_column, values=sortattribute, aggfunc=agg_func).fillna(0)
        
        if plot_type == 'line':
            ax = pivot_df.plot(kind='line', marker='o', title=title if title else "")
            plt.xlabel(x_column)
            plt.ylabel(sortattribute)
        elif plot_type == 'bar':
            ax = pivot_df.plot(kind='bar', stacked=True, title=title if title else "")
            plt.xlabel(x_column)
            plt.ylabel(sortattribute)
        else:
            print("Pivoted data only supports 'line' and 'bar' plot types.")
    else:
        # Standard plot handling
        if plot_type == 'bar':
            if y_column:
                ax = logs_df.groupby(x_column)[y_column].agg(agg_func).plot(kind='bar', color='skyblue')
            else:
                ax = logs_df[x_column].value_counts().plot(kind='bar', color='skyblue')
            plt.xlabel(x_column)
            plt.ylabel(sortattribute if not y_column else y_column)
        
        elif plot_type == 'line' and y_column:
            ax = logs_df.groupby(x_column)[y_column].agg(agg_func).plot(kind='line', marker='o')
            plt.xlabel(x_column)
            plt.ylabel(y_column)
        
        elif plot_type == 'scatter' and y_column:
            plt.scatter(logs_df[x_column], logs_df[y_column], alpha=0.5, color='purple')
            plt.xlabel(x_column)
            plt.ylabel(y_column)
            ax = plt.gca()
        
        elif plot_type == 'hist':
            ax = logs_df[x_column].plot(kind='hist', bins=15, color='skyblue', edgecolor='black')
            plt.xlabel(x_column)
            plt.ylabel('Frequency')
    
    # Set the plot title
    if title:
        plt.title(title)
    
    # Add individual count labels on each bar
    if plot_type == 'bar':
        for container in ax.containers:
            ax.bar_label(container, fmt='%d', label_type='center')

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
# Plot data as a table using matplotlib, with an option to select specific columns
def plot_table(df, columns=None, title="Table Plot", col_widths=None, fontsize=10):
    if columns:
        df = df[columns]
    
    plt.figure(figsize=(len(df.columns) * 2, len(df) * 0.4))
    ax = plt.gca()
    ax.axis('off')
    
    table = plt.table(cellText=df.values,
                      colLabels=df.columns,
                      cellLoc='center',
                      loc='center',
                      colWidths=col_widths or [0.2] * len(df.columns))
    
    table.auto_set_font_size(False)
    table.set_fontsize(fontsize)
    table.scale(1, 1.5)
    
    plt.title(title, fontsize=fontsize + 2)
    plt.tight_layout()
    plt.show()

## 5.1 Generated Log Data

### Log Count per Fault-Target Across Processing Stages

In [ ]:
# Rename count columns to represent each processing stage
collected_logs_count = count_logs(collected_logs_df, 'fault-target')
templated_logs_count = templated_logs_df.groupby('fault-target')['templated_message'].nunique().reset_index(name='count')
difference_logs_count = difference_logs_df.groupby('fault-target')['templated_message'].nunique().reset_index(name='count')
generalized_logs_count = count_logs(generalized_logs_df, 'fault-target')

collected_logs_count = collected_logs_count.rename(columns={'count': 'Preparation'})
templated_logs_count = templated_logs_count.rename(columns={'count': 'Generalization'})
difference_logs_count = difference_logs_count.rename(columns={'count': 'Difference'})
generalized_logs_count = generalized_logs_count.rename(columns={'count': 'Enrichment'})

# Merge the data based on 'fault-target' attribute
merged_counts = collected_logs_count.merge(templated_logs_count, on='fault-target', how='outer') \
                                    .merge(difference_logs_count, on='fault-target', how='outer') \
                                    .merge(generalized_logs_count, on='fault-target', how='outer')
                                    
# Fill NaN with 0 and sort by descending order
merged_counts = merged_counts.fillna(0)
merged_counts = merged_counts.sort_values(by='Enrichment', ascending=False)
print(merged_counts)

### Log Count per Component Across Processing Stages

In [ ]:
# Rename count columns to represent each processing stage
collected_logs_count_c = count_logs(collected_logs_df, 'component')
templated_logs_count_c = templated_logs_df.groupby('component')['templated_message'].nunique().reset_index(name='count')
difference_logs_count_c = difference_logs_df.groupby('component')['templated_message'].nunique().reset_index(name='count')
generalized_logs_count_c = count_logs(generalized_logs_df, 'component')

collected_logs_count_c = collected_logs_count_c.rename(columns={'count': 'Preparation'})
templated_logs_count_c = templated_logs_count_c.rename(columns={'count': 'Generalization'})
difference_logs_count_c = difference_logs_count_c.rename(columns={'count': 'Difference'})
generalized_logs_count_c = generalized_logs_count_c.rename(columns={'count': 'Enrichment'})

# Merge the data on 'component' attribute
merged_counts = collected_logs_count_c.merge(templated_logs_count_c, on='component', how='outer') \
                                    .merge(difference_logs_count_c, on='component', how='outer') \
                                    .merge(generalized_logs_count_c, on='component', how='outer')
                                    
# Fill NaN with 0 and sort by descending order
merged_counts = merged_counts.fillna(0)
merged_counts = merged_counts.sort_values(by='Enrichment', ascending=False)
print(merged_counts)

## 5.2 Fault Injection

### Log Count per Fault-Target per Component

In [ ]:
# Count number of faults using 'fault-target' and 'component' attributes
generalized_logs_fault_count = group_by_and_count(generalized_logs_df, 'fault-target', 'component')
generalized_logs_fault_count = generalized_logs_fault_count.rename(columns={'count': 'Log Count'})
generalized_logs_fault_count = generalized_logs_fault_count.sort_values(by='Log Count', ascending=False)

plot_logs_data(
    logs_df=generalized_logs_fault_count,
    sortattribute='Log Count',
    x_column='fault-target',
    y_column='component',
    plot_type='bar',
    title=None,
    pivot=True,
    agg_func='sum'
)
print(generalized_logs_fault_count)

### Log Count per Component per Fault-Scenario

In [ ]:
# Count number of faults using 'component' and 'fault-target' attributes
generalized_logs_component_count = group_by_and_count(generalized_logs_df,  'component', 'fault-target')
generalized_logs_component_count = generalized_logs_component_count.rename(columns={'count': 'Log Count'})
generalized_logs_component_count = generalized_logs_component_count.sort_values(by='Log Count', ascending=False)

plot_logs_data(
    logs_df=generalized_logs_component_count,
    sortattribute='Log Count',
    x_column='component',
    y_column='fault-target',
    plot_type='bar',
    title=None,
    pivot=True,
    agg_func='sum'
)
print(generalized_logs_component_count)

### Enriched Attribute Percentage per Fault-Scenario

In [ ]:
# Filter rows based on attributes
filtered_df = generalized_logs_df[
    (generalized_logs_df['component-specific'] == True) |
    (generalized_logs_df['fault-specific'] == True) |
    (generalized_logs_df['rare'] == True) |
    (generalized_logs_df['direct_cause'] == True) |
    (generalized_logs_df['phase'] == 'crash')
]

enriched_attributes = ['component-specific', 'fault-specific', 'phase', 'rare', 'direct_cause']
enriched_counts_t = pd.DataFrame()

# Calculate the count of previously filtered rows and group them using the 'fault-target' attribute
for attribute in enriched_attributes:
    if attribute == 'phase':
        attribute_count = filtered_df[filtered_df[attribute] == 'crash'].groupby('fault-target').size().reset_index(name= 'crash')
    else:
        attribute_count = filtered_df[filtered_df[attribute] == True].groupby('fault-target').size().reset_index(name=attribute)   
    if enriched_counts_t.empty:
        enriched_counts_t = attribute_count
    else:
        enriched_counts_t = enriched_counts_t.merge(attribute_count, on='fault-target', how='outer')

# Replace NaN values with 0
enriched_counts_t = enriched_counts_t.fillna(0)

# Calculate the total count of each fault-target, merge it with the enriched counts and sort by descending order
total_fault_target_count = generalized_logs_df.groupby('fault-target').size().reset_index(name='total_count')
enriched_counts_t_df = total_fault_target_count.merge(enriched_counts_t, on='fault-target', how='outer')
enriched_counts_t_df = enriched_counts_t_df.sort_values(by='total_count', ascending=False)

enriched_percentages_t = enriched_counts_t_df.copy()

# Calculate the percentage of each attribute
attributes_to_convert = ['component-specific', 'fault-specific', 'crash', 'rare', 'direct_cause']
for attribute in attributes_to_convert:
    enriched_percentages_t[f'{attribute}'] = ((enriched_percentages_t[attribute] / enriched_percentages_t['total_count']) * 100).round(1)
enriched_percentages_t = enriched_percentages_t[['fault-target', 'total_count'] + [f'{attr}' for attr in attributes_to_convert]]

print(enriched_percentages_t)


### Enriched Attribute Percentage per Component

In [ ]:
# Filter rows based on attributes
filtered_df = generalized_logs_df[
    (generalized_logs_df['component-specific'] == True) |
    (generalized_logs_df['fault-specific'] == True) |
    (generalized_logs_df['rare'] == True) |
    (generalized_logs_df['direct_cause'] == True) |
    (generalized_logs_df['phase'] == 'crash')
]

enriched_attributes = ['component-specific', 'fault-specific', 'phase', 'rare', 'direct_cause']
enriched_counts_r = pd.DataFrame()

# Calculate the count of previously filtered rows and group them using the 'crash' attribute
for attribute in enriched_attributes:
    if attribute == 'phase':
        attribute_count = filtered_df[filtered_df[attribute] == 'crash'].groupby('component').size().reset_index(name= 'crash')
    else:
        attribute_count = filtered_df[filtered_df[attribute] == True].groupby('component').size().reset_index(name=attribute)
    if enriched_counts_r.empty:
        enriched_counts_r = attribute_count
    else:
        enriched_counts_r = enriched_counts_r.merge(attribute_count, on='component', how='outer')

# Replace NaN values with 0
enriched_counts_r = enriched_counts_r.fillna(0)

# Calculate the total count of each component, merge it with the enriched counts and sort by descending order
total_fault_target_count = generalized_logs_df.groupby('component').size().reset_index(name='total_count')
enriched_counts_r_df = total_fault_target_count.merge(enriched_counts_r, on='component', how='outer')
enriched_counts_r_df = enriched_counts_r_df.sort_values(by='total_count', ascending=False)

enriched_percentages_r = enriched_counts_r_df.copy()

# Calculate the percentage of each attribute
attributes_to_convert = ['component-specific', 'fault-specific', 'crash', 'rare', 'direct_cause']
for attribute in attributes_to_convert:
    enriched_percentages_r[f'{attribute}'] = ((enriched_percentages_r[attribute] / enriched_percentages_r['total_count']) * 100).round(1)
enriched_percentages_r = enriched_percentages_r[['component', 'total_count'] + [f'{attr}' for attr in attributes_to_convert]]

print(enriched_percentages_r)

## 5.3 Anomaly determination Results

### Anomaly Count per Fault-Scenario

In [ ]:
# Filter for rows where 'determination' contains 'anomaly'
anomalies_df_f = determination_logs_df[determination_logs_df['determination'].str.contains('anomaly', case=False, na=False)]

# Group by 'fault-target' attribute and count the anomalies
anomaly_count_f = anomalies_df_f.groupby('fault-target').size().reset_index(name='anomaly_count')
anomaly_count_f = anomaly_count_f.sort_values(by='anomaly_count', ascending=False)

print(anomaly_count_f)

### Anomaly Count per Component

In [ ]:
# Filter for rows where 'determination' contains 'anomaly'
anomalies_df_c = determination_logs_df[determination_logs_df['determination'].str.contains('anomaly', case=False, na=False)]

# Group by 'fault-target' attribute and count the anomalies
anomaly_count_c = anomalies_df_c.groupby('component').size().reset_index(name='anomaly_count')
anomaly_count_c = anomaly_count_c.sort_values(by='anomaly_count', ascending=False)

print(anomaly_count_c)

In [ ]:
# Filter rows where 'determination' contains 'anomaly' in the determination_logs_df
anomalies_df = determination_logs_df[determination_logs_df['determination'].str.contains('anomaly', case=False, na=False)]

# Group by 'component' attribute and count the anomalies
anomaly_count_per_component = anomalies_df.groupby('component').size().reset_index(name='anomaly_count')

print(anomaly_count_per_component)

### Anomaly Count per fault-target per component

In [ ]:
# Count number of faults using 'component' and 'fault-target' attributes

anomalies_df = determination_logs_df[determination_logs_df['determination'].str.contains('anomaly', case=False, na=False)]
anomaly_distribution = anomalies_df.groupby(['fault-target', 'component']).size().reset_index(name='anomaly_count')
anomaly_distribution = anomaly_distribution.sort_values(by='anomaly_count', ascending=False)

plot_logs_data(
    logs_df=anomaly_distribution,      
    x_column='fault-target',           
    y_column='component',              
    plot_type='bar',                   
    title=None,
    pivot=True,                        
    agg_func='sum',                     
    sortattribute='anomaly_count'
)

print(anomaly_distribution)

### Anomaly Determination Counts and Attributes Percentages

In [ ]:
# Filter rows based on attributes
filtered_df = determination_logs_df[
    (determination_logs_df['component-specific'] == True) |
    (determination_logs_df['fault-specific'] == True) |
    (determination_logs_df['phase'] == 'crash') |
    (determination_logs_df['rare'] == True) |
    (determination_logs_df['direct_cause'] == True)
]

enriched_attributes = ['component-specific', 'fault-specific', 'phase', 'rare', 'direct_cause']
enriched_counts_r = pd.DataFrame()

# Calculate the count of previously filtered rows and group them using the 'crash' attribute
for attribute in enriched_attributes:
    if attribute == 'phase':
        attribute_count = filtered_df[filtered_df[attribute] == 'crash'].groupby('determination').size().reset_index(name= 'crash')
    else:
        attribute_count = filtered_df[filtered_df[attribute] == True].groupby('determination').size().reset_index(name=attribute)
    if enriched_counts_r.empty:
        enriched_counts_r = attribute_count
    else:
        enriched_counts_r = enriched_counts_r.merge(attribute_count, on='determination', how='outer')

# Replace NaN values with 0
enriched_counts_r = enriched_counts_r.fillna(0)

# Calculate the total count of each component, merge it with the enriched counts and sort by descending order
total_fault_target_count = determination_logs_df.groupby('determination').size().reset_index(name='total_count')
enriched_counts_r_df = total_fault_target_count.merge(enriched_counts_r, on='determination', how='outer')
enriched_counts_r_df = enriched_counts_r_df.sort_values(by='total_count', ascending=False)

enriched_percentages_r = enriched_counts_r_df.copy()

# Calculate the percentage for each enriched attribute by dividing by total_count and multiplying by 100
attributes_to_convert = ['component-specific', 'fault-specific', 'crash', 'rare', 'direct_cause']
for attribute in attributes_to_convert:
    enriched_percentages_r[f'{attribute}'] = ((enriched_percentages_r[attribute] / enriched_percentages_r['total_count']) * 100).round(1)
enriched_percentages_r = enriched_percentages_r[['determination', 'total_count'] + [f'{attr}' for attr in attributes_to_convert]]


print(enriched_percentages_r)

### Anomaly Determination Counts per Component

In [ ]:
def clean_data(df):
    # Perform aggregation grouped on columns: 'determination', 'component'
    df = df[df['determination'].str.contains("anomaly", regex=False, na=False, case=False)]
    df = df.groupby(['determination', 'component']).agg(determination_count=('determination', 'count')).reset_index()
    return df

df_clean = clean_data(determination_logs_df.copy())
df_clean = df_clean.sort_values(by='determination_count', ascending=False)

plot_logs_data(
    logs_df=df_clean,      
    x_column='determination',          
    y_column='component',             
    plot_type='bar',                  
    pivot=True,                       
    agg_func='sum',                   
    sortattribute='determination_count'
)

df_clean
print(df_clean)

### Anomaly Determination Counts per Fault-Target

In [ ]:
def clean_data(df):
    # Perform aggregation grouped on columns: 'determination', 'fault-target'
    df = df[df['determination'].str.contains("anomaly", regex=False, na=False, case=False)]
    df = df.groupby(['determination', 'fault-target']).agg(determination_count=('determination', 'count')).reset_index()
    return df

df_clean = clean_data(determination_logs_df.copy())
df_clean = df_clean.sort_values(by='determination_count', ascending=False)

plot_logs_data(
    logs_df=df_clean, 
    x_column='determination', 
    y_column='fault-target',  
    plot_type='bar',          
    pivot=True,               
    agg_func='sum',           
    sortattribute='determination_count'
)

print(df_clean)

### Direct Cause Anomalies

In [ ]:
def clean_data(df):
    # Filter rows based on columns: 'determination', 'direct_cause'
    df = df[(df['determination'].str.contains("anomaly", regex=False, na=False, case=False)) & (df['direct_cause'] == True)]
    return df

df_clean = clean_data(determination_logs_df.copy())
print(df_clean)